In [2]:
import sys
sys.path.append("/home/codespace/spark/spark-3.3.2-bin-hadoop3/python")
sys.path.append("/home/codespace/spark/spark-3.3.2-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip")

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/03/06 21:28:18 WARN Utils: Your hostname, codespaces-06e558 resolves to a loopback address: 127.0.0.1; using 10.0.0.30 instead (on interface eth0)
25/03/06 21:28:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/06 21:28:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
print(pyspark.__version__)

3.3.2


In [ ]:
#!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

In [6]:
!wc -l yellow_tripdata_2024-10.parquet

254534 yellow_tripdata_2024-10.parquet


In [7]:
df_yellow = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2024-10.parquet')

df_yellow.head(5)

[Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 10, 1, 0, 30, 44), tpep_dropoff_datetime=datetime.datetime(2024, 10, 1, 0, 48, 26), passenger_count=1, trip_distance=3.0, RatecodeID=1, store_and_fwd_flag='N', PULocationID=162, DOLocationID=246, payment_type=1, fare_amount=18.4, extra=1.0, mta_tax=0.5, tip_amount=1.5, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=24.9, congestion_surcharge=2.5, Airport_fee=0.0),
 Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2024, 10, 1, 0, 12, 20), tpep_dropoff_datetime=datetime.datetime(2024, 10, 1, 0, 25, 25), passenger_count=1, trip_distance=2.2, RatecodeID=1, store_and_fwd_flag='N', PULocationID=48, DOLocationID=236, payment_type=1, fare_amount=14.2, extra=3.5, mta_tax=0.5, tip_amount=3.8, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=23.0, congestion_surcharge=2.5, Airport_fee=0.0),
 Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2024, 10, 1, 0, 4, 46), tpep_dropoff_datetime=datetime.datetime(2

In [9]:
df_yellow = df_yellow.repartition(4)

df_yellow.write.parquet('data/pq/yellow/2024/10/')

In [11]:
df_yellow = spark.read.parquet('data/pq/yellow/2024/10/')

How many taxi trips were there on the 15th of October?

In [12]:
from pyspark.sql import functions as F

In [14]:
df_yellow \
    .withColumn('pickup_date', F.to_date(df_yellow.tpep_pickup_datetime)) \
    .filter("pickup_date = '2024-10-15'") \
    .count()

128893

Option 2 to count the number of trips on October 15

In [15]:
df_yellow.registerTempTable('yellow_2024_10')

/home/codespace/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [17]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    yellow_2024_10
WHERE
    to_date(tpep_pickup_datetime) = '2024-10-15';
""").show()

[Stage 10:>                                                         (0 + 2) / 2]

+--------+
|count(1)|
+--------+
|  128893|
+--------+



Q4: What is the length of the longest trip in the dataset in hours?

In [18]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [19]:
df_yellow \
    .withColumn('duration', df_yellow.tpep_dropoff_datetime.cast('long') - df_yellow.tpep_pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df_yellow.tpep_pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

[Stage 13:>                                                         (0 + 2) / 2]

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2024-10-16|       585424|
| 2024-10-03|       515970|
| 2024-10-22|       495938|
| 2024-10-18|       413405|
| 2024-10-21|       323634|
+-----------+-------------+



Option 2 with direct sql

In [21]:
spark.sql("""
SELECT
    to_date(tpep_pickup_datetime) AS pickup_date,
    MAX((CAST(tpep_dropoff_datetime AS LONG) - CAST(tpep_pickup_datetime AS LONG)) / 60 / 60) AS duration
FROM 
    yellow_2024_10
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

[Stage 19:>                                                         (0 + 2) / 2]

+-----------+------------------+
|pickup_date|          duration|
+-----------+------------------+
| 2024-10-16| 162.6177777777778|
| 2024-10-03|           143.325|
| 2024-10-22|137.76055555555556|
| 2024-10-18|114.83472222222221|
| 2024-10-21| 89.89833333333333|
| 2024-10-20| 89.44611111111111|
| 2024-10-12| 67.57333333333334|
| 2024-10-17| 66.06666666666666|
| 2024-10-24| 38.47416666666666|
| 2024-10-23| 33.95111111111111|
+-----------+------------------+



Question 6: Least frequent pickup location zone

In [22]:
df_zones = spark.read.parquet('zones')

In [23]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [24]:
df_zones.registerTempTable('zones')

In [25]:
spark.sql("""
SELECT
    pul.Zone,
    COUNT(1)
FROM 
    yellow_2024_10 yw LEFT JOIN zones pul ON yw.PULocationID = pul.LocationID
GROUP BY 
    1
ORDER BY
    2 ASC
LIMIT 5;
""").show()

[Stage 24:>                                                         (0 + 2) / 2]

+--------------------+--------+
|                Zone|count(1)|
+--------------------+--------+
|Governor's Island...|       1|
|       Rikers Island|       2|
|       Arden Heights|       2|
|         Jamaica Bay|       3|
| Green-Wood Cemetery|       3|
+--------------------+--------+

